In [1]:
from dask.distributed import Client

client = Client("tcp://127.0.0.1:38200")
client

Connection method: Direct,
Dashboard: http://127.0.0.1:8787/status,
Comm: tcp://127.0.0.1:38200,Workers: 0
Dashboard: http://127.0.0.1:8787/status,Total threads: 0
Started: 28 minutes ago,Total memory: 0 B


In [2]:
import xarray
import colorcet as cc
import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm
import matplotlib
import numpy as np
import os
import pandas as pd
import json
import seaborn
import yaml
import copy
from pyeosim.atmosphere import SixSV_atmosphere
from pyeosim.datasets import DATA_PATHS
from pyeosim.spectral import TreeView_3
from pyeosim.sensor import TdiCmos
from pyeosim.plot import rgb
from hsman import get_datasets, open_dataset
import Py6S

In [3]:
ds = open_dataset('OUMI1202006221_VNIR_aerial',
                  chunks={'band':200,'y':500, 'x':500}).isel(y=slice(7000,14500),
                                                     x=slice(10000,17500))

surface_reflectance = ds.swap_dims({'band':'wavelength'})
# slice between 400 and 898 to reduce data volume and align with the bandpass responses used later in simulation
# Also rescale to reflectance by divide by 10000
surface_reflectance = surface_reflectance.sel(wavelength=slice(400,898)).astype(float)/10000
# replace anything with a greater than 1 reflectance by truncating at 1
surface_reflectance = surface_reflectance.where(surface_reflectance <= 1, other=1)

In [4]:
surface_reflectance

,Array,Chunk
Bytes,64.12 GiB,291.82 MiB
Shape,"(153, 7500, 7500)","(153, 500, 500)"
Count,73935 Tasks,225 Chunks
Type,float64,numpy.ndarray


In [5]:
# PARAMETERS
ATMOSPHERE_PATH = os.path.join('atmospheres', 'atmospheres_2')
ATMOSPHERE_SCENARIO = 'mk_20200622_1030_nadir'
spectral_response = TreeView_3()

In [6]:
def generate_mk_test_sixs(scenario):
    a = Py6S.SixS()
    a.ground_reflectance = Py6S.GroundReflectance.HomogeneousLambertian(1)
    a.altitudes.set_sensor_satellite_level()
    geom = Py6S.Geometry.User()
    geom.solar_z=scenario['solar_z']
    geom.solar_a=scenario['solar_a']
    geom.view_z=scenario['view_z']
    geom.view_a=scenario['view_a']
    geom.day=scenario['day']
    geom.month=scenario['month']
    a.geometry = geom
    ac = Py6S.AtmosCorr()
    a.atmos_corr = ac.AtmosCorrLambertianFromReflectance(.5)
    return a

In [7]:
def read_atmos_meta(path):
    """
    Loads JSON file containing atmospheric simulation parameters used in 6SV
    """
    jsons = os.listdir(path)
    jsons = [x for x in os.listdir(path) if x.endswith('json')]
    metas = {}
    for _json in jsons:
        with open(os.path.join(path, _json), 'r') as f:
            metas[_json.split('.')[0]] = json.load(f)
    return pd.DataFrame(metas).T

atmos_meta = read_atmos_meta(ATMOSPHERE_PATH)
atmos_meta

,version,month,day,solar_z,solar_a,view_z,view_a,scattering_angle,azimuthal_angle_difference,visibility,...,atmospheric_intrinsic_reflectance,background_reflectance,pixel_reflectance,direct_solar_irradiance,diffuse_solar_irradiance,environmental_irradiance,atmospheric_intrinsic_radiance,background_radiance,pixel_radiance,solar_spectrum
inv_20200622_1030_nadir,1.1,6,22,36,150,0,0,143.36,150.3,8.49,...,0.089,0.384,0.537,639.951,601.212,306.191,41.59,180.168,252.17,1836.922
inv_20200622_1200_nadir,1.1,6,22,34,186,0,0,145.86,186.04,8.49,...,0.091,0.387,0.541,677.159,611.508,317.91,44.036,187.063,261.821,1836.922
inv_20200301_1030_nadir,1.1,3,1,67,156,0,0,112.95,156.93,8.49,...,0.125,0.309,0.432,135.397,374.7,125.839,29.904,74.046,103.638,1933.95
inv_20201001_1030_nadir,1.1,10,1,62,162,0,0,117.81,162.28,8.49,...,0.112,0.33,0.461,210.29,427.994,157.463,31.415,92.654,129.682,1892.893
inv_20200801_1030_nadir,1.1,8,1,42,151,0,0,137.72,151.28,8.49,...,0.088,0.377,0.528,551.236,575.304,277.914,38.208,163.529,228.882,1841.392
inv_20200501_1030_nadir,1.1,5,1,44,155,0,0,135.81,155.35,8.49,...,0.088,0.374,0.524,527.183,573.474,271.529,37.746,159.772,223.623,1870.334
mk_20200622_1030_nadir,1.1,6,22,33,140,0,0,146.71,140.71,8.49,...,0.091,0.387,0.542,689.322,614.763,321.714,44.657,189.301,264.954,1836.922
mk_20200622_1200_nadir,1.1,6,22,28,180,0,0,151.4,180.45,8.49,...,0.092,0.391,0.547,752.294,630.802,341.206,47.228,200.771,281.007,1836.922
mk_20200301_1030_nadir,1.1,3,1,63,152,0,0,116.99,152.18,8.49,...,0.113,0.326,0.457,200.697,427.417,154.954,31.679,91.177,127.615,1933.95
mk_20201001_1030_nadir,1.1,10,1,57,157,0,0,122.21,157.28,8.49,...,0.103,0.345,0.482,287.355,474.943,188.056,33.019,110.655,154.878,1892.893


In [8]:
atmos = SixSV_atmosphere(generate_mk_test_sixs(atmos_meta.loc[ATMOSPHERE_SCENARIO]))

In [9]:
atmos.fit(spectral_response)

In [10]:
# # generate Top of Atmosphere radiance array
radiance = atmos.transform(surface_reflectance)

In [11]:
sensor_configs = yaml.load(open('sensors.yaml', 'r'), Loader=yaml.Loader)
print('Available Sensor Configurations:')
list(sensor_configs.keys())

Available Sensor Configurations:


['TreeView_1_MSI_16',
 'TreeView_1_MSI_32',
 'TreeView_2_MSI_16',
 'TreeView_2_MSI_32',
 'TreeView_1_PAN_32',
 'TreeView_1_PAN_64',
 'TreeView_2_PAN_32',
 'TreeView_2_PAN_64',
 'TreeView_3_MSI_16',
 'TreeView_3_MSI_32',
 'TreeView_3_PAN_32',
 'TreeView_3_PAN_64']

In [12]:
msi_sensor = TdiCmos(**sensor_configs['TreeView_3_MSI_32'])
msi_sensor_simulation = msi_sensor.fit_transform(radiance).compute()

ValueError: dimension y on 0th function argument to apply_ufunc with dask='parallelized' consists of multiple chunks, but is also a core dimension. To fix, either rechunk into a single dask array chunk along this dimension, i.e., ``.chunk(dict(y=-1))``, or pass ``allow_rechunk=True`` in ``dask_gufunc_kwargs`` but beware that this may significantly increase memory usage.

In [36]:
msi_sensor_simulation.astype('uint16').to_netcdf('medium_treeview_test_data.nc')